# Data preparation

## Import Libs

In [1]:
%matplotlib inline
import numpy as np
from utils.helper import read_data, morphology
from tensorflow import keras
FOLDER = "data/data_big/"
MORPHOLOGY_ACTIVATED = True

## Load data

In [2]:
data_empty = read_data(FOLDER + "empty.csv")
data_1p = read_data(FOLDER + "1p.csv")
data_2p = read_data(FOLDER + "2p.csv")
data_3p = read_data(FOLDER + "3p.csv")
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

----------------Shapes----------------
empty: (480, 5, 3, 64, 64)
1p: (480, 5, 3, 64, 64)
2p: (480, 5, 3, 64, 64)
3p: (480, 5, 3, 64, 64)


In [3]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

In [4]:
data_empty_summed = np.abs(data_empty) 
data_1p_summed = np.abs(data_1p)
data_2p_summed = np.abs(data_2p)
data_3p_summed = np.abs(data_3p)

In [5]:
data_empty_summed[:, :, :, 32, :] = 0
data_1p_summed[:, :, :, 32, :] = 0
data_2p_summed[:, :, :, 32, :] = 0
data_3p_summed[:, :, :, 32, :] = 0

In [6]:
data_0 = np.moveaxis(data_empty_summed, 2,4)
data_1 = np.moveaxis(data_1p_summed, 2,4)
data_2 = np.moveaxis(data_2p_summed, 2,4)
data_3 = np.moveaxis(data_3p_summed, 2,4)
print(data_0.shape)
print(data_1.shape)
print(data_2.shape)
print(data_3.shape)

(480, 5, 64, 64, 3)
(480, 5, 64, 64, 3)
(480, 5, 64, 64, 3)
(480, 5, 64, 64, 3)


In [7]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
data_tot.shape

(1920, 5, 64, 64, 3)

In [8]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

Labels shape: (1920,)


In [9]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

Labels abels one hot shape: (1920, 4)


In [10]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot, train_labels_one_hot, test_size=0.2, stratify = train_labels_one_hot)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

Shape train_X: (1536, 5, 64, 64, 3)
Shape test_X: (384, 5, 64, 64, 3)
Shape train_y: (1536, 4)
Shape test_y: (384, 4)


# Model

In [11]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(5, 64, 64, 3)))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(8,
                      input_shape = (64,64,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(2,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4,
                      input_shape = (32,32,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(2,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(4,
                      input_shape = (16,16,3),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu")))
model.add(keras.layers.TimeDistributed(keras.layers.MaxPooling2D(2,
                                strides = None,
                                padding = "valid")))
model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
# define LSTM model
model.add(keras.layers.LSTM(5))
model.add(keras.layers.Dense(4))
model.add(keras.layers.Activation("softmax"))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 5, 64, 64, 8)     608       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 5, 32, 32, 8)     0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 5, 32, 32, 4)     804       
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 5, 16, 16, 4)     0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 5, 16, 16, 4)     4

In [12]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 100,
    verbose = 1,
    validation_split = 0.1,
    shuffle= True,
    callbacks=[keras.callbacks.EarlyStopping(patience = 3)]
)

model.save("models_kosta/LSTM")

Epoch 1/100
44/44 [==============================] - 15s 287ms/step - loss: 1.2057 - accuracy: 0.4421 - val_loss: 0.9090 - val_accuracy: 0.4740
Epoch 2/100
44/44 [==============================] - 12s 275ms/step - loss: 0.8474 - accuracy: 0.5550 - val_loss: 0.7673 - val_accuracy: 0.6429
Epoch 3/100
44/44 [==============================] - 12s 275ms/step - loss: 0.7597 - accuracy: 0.6585 - val_loss: 0.7196 - val_accuracy: 0.7078
Epoch 4/100
44/44 [==============================] - 13s 292ms/step - loss: 0.7119 - accuracy: 0.6860 - val_loss: 0.6988 - val_accuracy: 0.7273
Epoch 5/100
44/44 [==============================] - 12s 273ms/step - loss: 0.6970 - accuracy: 0.6896 - val_loss: 0.6706 - val_accuracy: 0.6883
Epoch 6/100
44/44 [==============================] - 12s 276ms/step - loss: 0.6541 - accuracy: 0.6954 - val_loss: 0.6616 - val_accuracy: 0.7208
Epoch 7/100
44/44 [==============================] - 12s 276ms/step - loss: 0.6357 - accuracy: 0.7149 - val_loss: 0.6299 - val_accuracy:

INFO:tensorflow:Assets written to: models_kosta/LSTM\assets


INFO:tensorflow:Assets written to: models_kosta/LSTM\assets


In [13]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

12/12 [==============================] - 1s 101ms/step - loss: 0.5630 - accuracy: 0.7057
Loss: 0.562972366809845
Accuracy: 0.7057291865348816
